<a href="https://colab.research.google.com/github/AhmedHussKhalifa/DPR/blob/master/DPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ! git clone https://github.com/facebookresearch/DPR
! git clone https://github.com/AhmedHussKhalifa/DPR
! python --version # now returns Python 3.6.5 :: Anaconda, Inc.

fatal: destination path 'DPR' already exists and is not an empty directory.
Python 3.6.9


In [ ]:
%%shell
cd DPR
pip install .

Processing /content/DPR
  Created wheel for dpr: filename=dpr-0.1.0-cp36-none-any.whl size=12891 sha256=c49b7c0940e0b9cf7de28f3f1d460f514b6bdf5c33a718e4b57a4e0c19e65829
  Stored in directory: /tmp/pip-ephem-wheel-cache-g3v329j0/wheels/7b/1b/8b/cbf33220fb7d3eb120e18f746a5b752ebc53f8523fb2eb0681
Successfully built dpr
  Found existing installation: dpr 0.1.0
    Uninstalling dpr-0.1.0:
      Successfully uninstalled dpr-0.1.0


In [ ]:
# download NQ datasets
%%shell
cd DPR
python data/download_data.py --resource data.retriever.qas.nq

python data/download_data.py --resource data.retriever.nq

# mkdir data/retriever_1
cd data/retriever
wget https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz -O nq-dev.json.gz
gzip -d nq-dev.json.gz
ls -sh nq-dev.json

# commands below download the full wikipedia dataset and nq-train datasets, which are
# too huge to load. I leave them here in case anyone wants to try the full datasets.
# We can use nq-train-subset.json and psgs_w100_subset.tsv to test DPR.

# cd data/wikipedia_split
# wget https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz 
# gzip -d psgs_w100.tsv.gz

# cd ../retriever
# wget https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-train.json.gz -O nq-train.json.gz
# gzip -d nq-train.json.gz
# ls -sh nq-train.json

Loading from  https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-dev.qa.csv
File already exist  ./data/retriever/qas/nq-dev.csv
Loading from  https://dl.fbaipublicfiles.com/dpr/nq_license/LICENSE
File already exist  ./data/retriever/qas/LICENSE
Loading from  https://dl.fbaipublicfiles.com/dpr/nq_license/README
File already exist  ./data/retriever/qas/README
Loading from  https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-test.qa.csv
File already exist  ./data/retriever/qas/nq-test.csv
Loading from  https://dl.fbaipublicfiles.com/dpr/nq_license/LICENSE
File already exist  ./data/retriever/qas/LICENSE
Loading from  https://dl.fbaipublicfiles.com/dpr/nq_license/README
File already exist  ./data/retriever/qas/README
Loading from  https://dl.fbaipublicfiles.com/dpr/data/retriever/nq-train.qa.csv
File already exist  ./data/retriever/qas/nq-train.csv
Loading from  https://dl.fbaipublicfiles.com/dpr/nq_license/LICENSE
File already exist  ./data/retriever/qas/LICENSE
Loading from  https://d

In [ ]:
%%shell
# THis for Downloading all the resourses
# Do not run that 
cd DPR
# python data/download_data.py --resource data

In [ ]:
%%shell

cd DPR

python data/download_data.py --resource data.retriever.nq-train 

python data/download_data.py --resource data.retriever.nq-dev

In [ ]:
%%shell

cd DPR

python -m torch.distributed.launch \
	--nproc_per_node=1 train_dense_encoder.py \
	--max_grad_norm 2.0 \
	--encoder_model_type hf_bert \
	--pretrained_model_cfg bert-base-uncased \
	--seed 12345 \
	--sequence_length 256 \
	--warmup_steps 1237 \
	--batch_size 2 \
	--do_lower_case \
	--train_file "/content/DPR/data/retriever/nq-train.tmp" \
	--dev_file "/content/DPR/data/retriever/nq-dev.json" \
	--output_dir "/content/DPR/output" \
	--learning_rate 2e-05 \
	--num_train_epochs 40 \
	--dev_batch_size 16 \
  --val_av_rank_start_epoch 1

In [ ]:
%%shell

cd DPR

# python data/download_data.py --resource data.retriever_results.nq.single.wikipedia_passages

python generate_dense_embeddings.py \
	--model_file "/content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919" \
	--ctx_file "/content/DPR/data/retriever_results/nq/single/wikipedia_passages_0.pkl" \
	--shard_id 0 \
  --num_shards 1 \
	--out_file "inference"

# python generate_dense_embeddings.py \
#   --model_file "/content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919" \
#   --ctx_file "/content/drive/My Drive/STAT946/DPR/psgs_w100_subset.tsv" \
#   --shard_id 0 \
#   --num_shards 1 \
#   --out_file inference

no resources found for specified key
Initialized host 7f048dd76ae9 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
Reading saved model from /content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919
Traceback (most recent call last):
  File "generate_dense_embeddings.py", line 144, in <module>
    main(args)
  File "generate_dense_embeddings.py", line 74, in main
    saved_state = load_states_from_checkpoint(args.model_file)
  File "/content/DPR/dpr/utils/model_utils.py", line 140, in load_states_from_checkpoint
    state_dict = torch.load(model_file, map_location=lambda s, l: default_restore_location(s, 'cpu'))
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.6/dist-packages/torch/serializat

CalledProcessError: ignored

In [ ]:
%%shell
# mkdir collections/msmarco-doc

# wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz -P collections/msmarco-doc

mkdir: cannot create directory ‘collections/msmarco-doc’: No such file or directory
--2020-11-09 08:12:42--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.trec.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 40.112.152.16
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|40.112.152.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8501799926 (7.9G) [application/x-gzip]
Saving to: ‘collections/msmarco-doc/msmarco-docs.trec.gz’

msmarco-docs.trec.g 100%[===================>]   7.92G  23.0MB/s    in 6m 18s  

2020-11-09 08:19:01 (21.5 MB/s) - ‘collections/msmarco-doc/msmarco-docs.trec.gz’ saved [8501799926/8501799926]



In [ ]:
# Training on nq-train-subset.json
# Dont forget to change the paths based on your directory structure
%%shell
cd DPR
# ls -sh "/content/drive/My Drive/nq-train-subset.json"
ls -sh "/content/drive/My Drive/STAT946/DPR/nq-train-subset.json"
ls "/content/drive/My Drive/STAT946/DPR/output/"

220M '/content/drive/My Drive/STAT946/DPR/nq-train-subset.json'


In [ ]:
%%shell
cd DPR
ls -sh "/content/drive/My Drive/STAT946/DPR/nq-train-subset.json"
ls -sh "data/retriever/nq-dev.json" 
[ -d "/content/drive/My Drive/STAT946/DPR/output/" ] && \
echo "Directory /content/drive/My Drive/STAT946/DPR/output/ exists."

python train_dense_encoder.py \
  --encoder_model_type hf_bert \
  --pretrained_model_cfg bert-base-uncased \
  --train_file "/content/drive/My Drive/STAT946/DPR/nq-train-subset.json"\
  --dev_file "data/retriever/nq-dev.json" \
  --output_dir "/content/drive/My Drive/STAT946/DPR/output"



220M '/content/drive/My Drive/STAT946/DPR/nq-train-subset.json'
778M data/retriever/nq-dev.json
Directory /content/drive/My Drive/STAT946/DPR/output/ exists.
Initialized host ab29005f0866 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
 **************** CONFIGURATION **************** 
adam_betas                     -->   (0.9, 0.999)
adam_eps                       -->   1e-08
batch_size                     -->   2
checkpoint_file_name           -->   dpr_biencoder
dev_batch_size                 -->   4
dev_file                       -->   data/retriever/nq-dev.json
device                         -->   cuda
distributed_world_size         -->   1
do_lower_case                  -->   False
dropout                        -->   0.1
encoder_model_type             -->   hf_bert
eval_per_epoch                 -->   1
fix_ctx_encoder                -->   False
fp16                           -->   False
fp16_opt_level                 -->   O1
global_loss_buf_sz       

CalledProcessError: ignored

In [ ]:
%%shell

cd DPR
ls -sh "/content/drive/My Drive/STAT946/DPR/psgs_w100_subset.tsv"
ls -sh "/content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919"
python generate_dense_embeddings.py \
  --model_file "/content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919" \
  --ctx_file "/content/drive/My Drive/STAT946/DPR/psgs_w100_subset.tsv" \
  --shard_id 0 \ 
  --num_shards 1 \
  --out_file inference

/bin/bash: line 3: cd: data/retriever: No such file or directory
--2020-11-09 18:16:19--  https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256239282 (244M) [application/gzip]
Saving to: ‘nq-dev.json.gz’

nq-dev.json.gz      100%[===================>] 244.37M  12.0MB/s    in 22s     

2020-11-09 18:16:41 (11.3 MB/s) - ‘nq-dev.json.gz’ saved [256239282/256239282]

778M nq-dev.json
1.3M '/content/drive/My Drive/STAT946/DPR/psgs_w100_subset.tsv'
ls: cannot access '/content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919': No such file or directory
usage: generate_dense_embeddings.py [-h]
                                    [--pretrained_model_cfg PRETRAINED_MODEL_CFG]
                                    [-

CalledProcessError: ignored

In [ ]:
%%shell

cd DPR

python dense_retriever.py \
  --model_file "/content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919" \
  --ctx_file "/content/drive/My Drive/STAT946/DPR/psgs_w100_subset.tsv" \
  --qa_file data/retriever/qas/nq-test.csv \
  --encoded_ctx_file inference_0.pkl \
  --out_file result \
  --n-docs 10

Initialized host ab29005f0866 as d.rank -1 on device=cuda, n_gpu=1, world size=1
16-bits training: False 
 **************** CONFIGURATION **************** 
batch_size                     -->   32
ctx_file                       -->   /content/drive/My Drive/STAT946/DPR/psgs_w100_subset.tsv
device                         -->   cuda
distributed_world_size         -->   1
do_lower_case                  -->   False
encoded_ctx_file               -->   inference_0.pkl
encoder_model_type             -->   None
fp16                           -->   False
fp16_opt_level                 -->   O1
hnsw_index                     -->   False
index_buffer                   -->   50000
local_rank                     -->   -1
match                          -->   string
model_file                     -->   /content/drive/My Drive/STAT946/DPR/output/dpr_biencoder.0.919
n_docs                         -->   10
n_gpu                          -->   1
no_cuda                        -->   False
out_file        

In [ ]:
%%shell

zip -r DPR.zip "/content/DPR"

  adding: content/DPR/ (stored 0%)
  adding: content/DPR/CHANGELOG.md (stored 0%)
  adding: content/DPR/train_reader.py (deflated 75%)
  adding: content/DPR/data/ (stored 0%)
  adding: content/DPR/data/download_data.py (deflated 84%)
  adding: content/DPR/data/retriever/ (stored 0%)
  adding: content/DPR/data/retriever/qas/ (stored 0%)
  adding: content/DPR/data/retriever/qas/nq-dev.csv (deflated 61%)
  adding: content/DPR/data/retriever/qas/README (deflated 37%)
  adding: content/DPR/data/retriever/qas/nq-train.csv (deflated 61%)
  adding: content/DPR/data/retriever/qas/nq-test.csv (deflated 60%)
  adding: content/DPR/data/retriever/qas/LICENSE (deflated 67%)
  adding: content/DPR/data/retriever/nq-dev.json (deflated 69%)
  adding: content/DPR/.git/ (stored 0%)
  adding: content/DPR/.git/info/ (stored 0%)
  adding: content/DPR/.git/info/exclude (deflated 28%)
  adding: content/DPR/.git/branches/ (stored 0%)
  adding: content/DPR/.git/packed-refs (deflated 37%)
  adding: content/DPR/.g

In [ ]:
%%shell
cd DPR
python data/download_data.py --resource data.retriever.trivia-train



/bin/bash: line 0: cd: DPR: No such file or directory
python3: can't open file 'data/download_data.py': [Errno 2] No such file or directory


CalledProcessError: ignored